# Übungsblatt 5 #

### Alice Ziegler, Daniel Schneider ###

## Aufgabe 5.1: Regression ##

#### Laden und Vorbereiten des `Wilt`-Datensatzes ####

In [2]:
import pandas as pd
import numpy as np
import math

In [3]:
training = pd.read_csv("wilt/training.csv")
testing = pd.read_csv("wilt/testing.csv")

In [4]:
training['class'] = training['class'].astype(dtype="category")
testing['class'] = testing['class'].astype(dtype="category")

In [5]:
training.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4339 entries, 0 to 4338
Data columns (total 6 columns):
class         4339 non-null category
GLCM_pan      4339 non-null float64
Mean_Green    4339 non-null float64
Mean_Red      4339 non-null float64
Mean_NIR      4339 non-null float64
SD_pan        4339 non-null float64
dtypes: category(1), float64(5)
memory usage: 173.8 KB


In [6]:
testing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 6 columns):
class         500 non-null category
GLCM_pan      500 non-null float64
Mean_Green    500 non-null float64
Mean_Red      500 non-null float64
Mean_NIR      500 non-null float64
SD_pan        500 non-null float64
dtypes: category(1), float64(5)
memory usage: 20.1 KB


### a) Lernen einer Regressionsfunktion ###

In [69]:
def regression(data,targetAttribute):
            
    t = data[[targetAttribute]].as_matrix()
    #print(t)
    
    D = data.drop([targetAttribute],axis=1).as_matrix()
    ones = np.ones((D.shape[0]))
    #print(ones)
    
    D = np.c_[ones,D]
    #print(D)
    
    DT = np.transpose(D)
    
    w = np.dot(np.dot(np.linalg.inv(np.dot(DT, D)), DT), t)
    #print(w)
    return (w)

In [70]:
def calculateValues(data, ws):
    Xs = data.as_matrix()
    ones = np.ones((Xs.shape[0]))
    
    Xs = np.c_[ones,Xs]
        
    t = np.dot(Xs, ws)
    #print(t)
    return(t)

In [71]:
w1 = regression(training.iloc[:,1:6],'SD_pan')

In [72]:
ts = testing[['SD_pan']].as_matrix()

In [73]:
tsCalc = calculateValues(testing.iloc[:,1:5],w1)

### b) MDS ###

In [63]:
def regressionWithClasses(data,targetAttribute,classAttribute):
    
    datas = groupByClasses(data,classAttribute)
    w = [None]*len(datas)
    
    for i in range(0,len(datas)):        
        w[i] = regression(datas[i].drop([classAttribute],axis=1),targetAttribute)
    
    return (w)

In [60]:
w2 = regressionWithClasses(training.iloc[:,0:6],'SD_pan','class')

In [62]:
def groupByClasses(data,classAttribute):
    cats = data[classAttribute].cat.categories

    dataClass = [None]*len(cats)
    
    for i in range(0,len(cats)):
        dataClass[i] = data.loc[testing[classAttribute] == cats[i]]
    
    return(dataClass)

In [65]:
testingClasses = groupByClasses(testing,'class')

In [85]:
tsClasses = [None]*len(testingClasses)
tsCalcClasses = [None]*len(testingClasses)

for i in range(0,len(testingClasses)):
    tsClasses[i] = testingClasses[i][['SD_pan']].as_matrix()
    tsCalcClasses[i] = calculateValues(testingClasses[i].iloc[:,1:5],w2[i])

### c) Stress ###

In [82]:
def squareError(tLearned,tOrig):
    err = 0.5*np.sum(np.square(tLearned-tOrig))
    print(err)

In [83]:
squareError(tsCalc,ts)

16676.6852235


In [84]:
for i in range(0,len(testingClasses)):
    squareError(tsCalcClasses[i],tsClasses[i])

12323.7864823
2396.73156755


## Aufgabe 4.2: Distanzmaße ##

### a) Beweis: Gemischte Distanzfunktion ist eine Metrik ###

Sei $p$ die Anzahl der Attribute unterschiedlichen Typs.
 
Zu zeigen: Die Funktion $ d: X \times X \to \mathbb {R} $ mit

$$
d(x_i,x_j) = \frac{\sum_{f=1}^p \delta_{ij}^{(f)} d_{ij}^{(f)}} {\sum_{f=1}^p \delta_{ij}^{(f)}}
$$

ist eine Metrik, wenn $d_{ij}^{(f)}$ metrisch ist.



Es müssen dafür die folgenden 3 Axiome gelten:

1. Positive Definitheit: 
$d (x_i, x_j) \geq 0$ und $d (x_i, x_j) = 0 \Leftrightarrow x_i = x_j$

2. Symmetrie: 
$d (x_i, x_j) = d (x_j, x_i)$

3. Dreiecksungleichung: 
$d (x_i, x_j) \leq d (x_i, x_k) + d (x_k, x_j)$

Es sei $d_{ij}^{(f)}$ eine Metrik. Dann dürfen keine $x_{if}$ undefiniert sein, da alle Distanzen bestimmbar sein müssen.

**Zu 1. **

Zu zeigen: $d (x_i, x_j) \geq 0$

* Nach Voraussetzung gilt: $d_{ij}^{(f)} \geq 0$ für alle $f \in \{1,...,p\}$.

  Für den binären Indikator $\delta_{ij}^{(f)}$ gilt nach Definition: 

     $\delta_{ij}^{(f)} \in {0,1}$ für alle $i,j \in \{1,...,n\}$ und $f \in \{1,...,p\}$, also $\delta_{ij}^{(f)} \geq 0$

   Daraus folgt: 
   $$
   \frac{\sum_{f=1}^p \delta_{ij}^{(f)} d_{ij}^{(f)}} {\sum_{f=1}^p \delta_{ij}^{(f)}} \geq 0 \Leftrightarrow d (x_i, x_j) \geq 0
   $$ 


Noch zu zeigen: $d (x_i, x_j) = 0 \Leftrightarrow x_i = x_j$. Zeige dazu beide Richtungen:

* $\Leftarrow$

  Wenn $x_i = x_j$, dann folgt, weil $d_{ij}^{(f)}$ metrisch ist, dass $d_{ij}^{(f)} = 0$ für alle $f \in \{1,...,p\}$ und daraus folgt: $\frac{\sum_{f=1}^p \delta_{ij}^{(f)} d_{ij}^{(f)}} {\sum_{f=1}^p \delta_{ij}^{(f)}} = 0 \Leftrightarrow d (x_i, x_j) = 0$ 


* $\Rightarrow$

  Sei nun $d (x_i, x_j) = 0$, dann muss $\sum_{f=1}^p \delta_{ij}^{(f)} d_{ij}^{(f)} = 0$ sein. Betrachte zwei Fälle:

  1. Falls $d_{ij}^{(f)} = 0$ für alle $f \in \{1,...,p\}$ gilt nach Voraussetzung $x_i = x_j$.

  2. Angenommen es gäbe ein $f_x \in \{1,...,p\}$ sodass $d_{ij}^{(f_x)} \neq 0$. Dann folgt daraus, dass $\delta_{ij}^{(f_x)} = 0$ sein muss. Dies ist allerdings nur der Fall, wenn $x_{if_x} = x_{jf_x}$ (für ein binäres asymetrisches Attribut $f_x$). Und dann gilt weiterhin $x_i = x_j$.


* Zusammen gilt also: 
$$
d (x_i, x_j) = 0 \Leftrightarrow x_i = x_j
$$

**Zu 2.**

Zu zeigen: $d (x_i, x_j) = d (x_j, x_i)$

* Nach Voraussetzung gilt: $d_{ij}^f = d_{ji}^f$ für alle $f \in \{1,...,p\}$

  Und auch für $\delta_{ij}^{(f)}$ gilt nach Definition, dass $\delta_{ij}^{(f)} = \delta_{ji}^{(f)}$ für alle $i,j \in \{1,...,n\}$ und $f \in \{1,...,p\}$

  Also gilt auch: $\sum_{f=1}^p \delta_{ij}^{(f)} d_{ij}^{(f)} = \sum_{f=1}^p \delta_{ij}^{(f)} d_{ji}^{(f)}$
  
  Und daraus folgt: 
  $$
  \frac{\sum_{f=1}^p \delta_{ij}^{(f)} d_{ij}^{(f)}} {\sum_{f=1}^p \delta_{ij}^{(f)}} = \frac{\sum_{f=1}^p \delta_{ji}^{(f)} d_{ji}^{(f)}} {\sum_{f=1}^p \delta_{ji}^{(f)}} \Leftrightarrow d (x_i, x_j) = d (x_j, x_i)
  $$


**Zu 3.**

Zu zeigen: $d (x_i, x_j) \leq d (x_i, x_k) + d (x_k, x_j)$

* Nach Voraussetzung gilt: $d_{ij}^f \leq d_{ik}^f + d_{kj}^f$ für alle $f \in \{1,...,p\}$
  
  Für $\delta_{ij}^{(f)}$ wiederum gilt:
  
  Falls $\delta_{ik}^{(f)} = 0$ oder $\delta_{kj}^{(f)} = 0$ folgt daraus, dass auch $\delta_{ij}^{(f)} = 0$ sein muss, da die Null nur angenommen wird, wenn zwei Attribute übereinstimmen. Es gilt also (da $\delta_{ik}^{(f)} \in \{0,1\}$) : $\delta_{ij}^{(f)} \leq \delta_{ik}^{(f)} + \delta_{kj}^{(f)}$.
  
  Und daraus folgt:
  $$
  \frac{\sum_{f=1}^p \delta_{ij}^{(f)} d_{ij}^{(f)}} {\sum_{f=1}^p \delta_{ij}^{(f)}} \leq 
  \frac{\sum_{f=1}^p \delta_{ik}^{(f)} d_{ik}^{(f)}} {\sum_{f=1}^p \delta_{ik}^{(f)}} + \frac{\sum_{f=1}^p \delta_{kj}^{(f)} d_{kj}^{(f)}} {\sum_{f=1}^p \delta_{kj}^{(f)}}
  \Leftrightarrow
  d (x_i, x_j) \leq d (x_i, x_k) + d (x_k, x_j)
  $$

### b) Speicherkomplexität von Distanzmatrizen ###

Die Speicherkomplexität der Distanzmatrix für $n$ Werte beträgt $O(n^2)$

Durch die symmetrie der Distanzen entspricht die Distanz von $x_i$ nach $x_j$ genau der von $x_j$ nach $x_i$ ($d_{ij}=d_{ji}$). 

Daher steht jeder Distanzwert zwei Mal in der Matrix, einmal an Position $i,j$ und noch einmal an Position $j,i$. Man könnte zwar jeden Wert auch nur einmal speichern, also nur die Position $i,j$ belegen und die Position $j,i$ leer lassen, dadurch wird allerdings die Matrix nicht kleiner, sie enthält nur weniger Werte (mehr NULL-Werte). Den Speicherverbrauch reduziert diese Lösung also nicht wirklich.

Um wirklich Speicherplatz einzusparen müsste man also eine andere Datenstruktur als eine Matrix wählen, sodass die doppelten Werte nur einmal gespeichert werden müssen, aber auch keine leeren Felder entstehen.

Beispielsweise könnte man eine Liste mit Dreiertupeln verwenden, sodass jedes Tupel eine Distanz enthält und zu jedem Distanzeintrag die beiden betreffenden Punkte (Start und Ende). Eine solche Lösung kann allerdings zu anderen Problemen führen: Wenn beispielsweise ein durchgängier Pfad verfolgt werden soll, zm dessen Distanz aufzuaddieren, ist es sehr viel aufwändiger, zu einem Endpunkt wieder den nächsten Startpunkt zu ermitteln, da jetzt jedes Mal zwei Spalten mit möglichen Punkten durchsucht werden müssen.

## Aufgabe 4.3: Verzerrte Werte ##

### a) Nicht-Erwartungstreue des Schätzers der Varianz ###

$$
\begin{align}
Wiederlegung\ Erwartungstreue\ Varianz: \\
s^2 = Var(x_i) &= {\frac{{\sum_{i=1}^N{(x_i-µ)^2}}}{N}} \\
E(Var(x_i)) &= E({\frac{{\sum_{i=1}^N{(x_i-µ)^2}}}{N}}) \\
&= {\frac{1}{N}}E({\sum_{i=1}^N{(x_i-µ)^2}})\\
&= {\frac{1}{N}} {\sum_{i=1}^N{(E(x_i^2)-2E(x_iµ)+E(µ^2))}} \\
&= {\frac{1}{N}}{\sum_{i=1}^N{((Var(x_i)+µ^2)\ (siehe\ I)\\ 
- 2({\frac{Var(x_i)}{N}}+µ^2)\  (siehe\ II)\\
+ {\frac{Var(x_i)}{N}} + µ^2)}}\ (siehe\ III)\\
&= {\frac{1}{N}}{\sum_{i=1}^N{((Var(x_i)+µ^2) - 2({\frac{Var(x_i)}{N}}+µ^2)+{\frac{Var(x_i}{N}}+µ^2)}} \\
&= {\frac{1}{N}}{\sum_{i=1}^N{(Var(x_i)+µ^2-{\frac{2Var(x_i)}{N}}-2µ^2 + {\frac{Var(x_i)}{N}} + µ^2)}} \\
&= {\frac{1}{N}}{\sum_{i=1}^N{(Var(x_i)-{\frac{2Var(x_i)}{N}}+ {\frac{Var(x_i)}{N}})}} \\
&= {\frac{1}{N}}{\sum_{i=1}^N({Var(x_i)-{\frac{Var(x_i)}{N}})}} \\
&= {\frac{1}{N}}(N Var(x_i) - {\frac{N Var(x_i}{N}} \\
&= Var(x_i) - {\frac{Var(x_i)}{N}} \\
&= Var(x_i) (1-{\frac{1}{N}}) \\
&= Var(x_i) ({\frac{N-1}{N}}) \\
\\
bias &= Var(x_i) (1-{\frac{1}{N}} - Var(x_i) \\
bias &= Var(x_i) - {\frac{Var(x_i)}{N}}- Var(x_i) \\
bias &= - {\frac{Var(x_i)}{N}} \\
bias &\neq 0 ==> nicht\ Erwartungstreu
\\
\\
Grundlage\ I: \\
(Verschiebungssatz) \\
Var(X_i) &= E(X_i^2) - E(X_i)^2 \\
E(X_i^2) &= Var(X_i) + E(X_i)^2 \\
wobei: \\ 
E(X_i)^2 &= µ^2 \\
\\
Grundlage\ II: \\
E(x_iµ) &= \frac{1}{N}{\sum_{j=1}^N{E(x_ix_j)}} \\
&= \frac{1}{N}E(x_i^2)+{\frac{1}{N}}{\sum_{j=1...N;\ (j\neq i)}E(x_i)E(x_j)} \\
&= \frac{1}{N}(Var(x_i) + E(x_i)^2) + \frac{1}{N}µ(N-1)µ \\
&= \frac{1}{N}(Var(x_i) + µ^2) + \frac{1}{N}(N-1)µ^2 \\
&= {\frac{Var(x_i)+µ^2+Nµ^2-µ^2}{N}} \\
&= {\frac{Var(x_i)}{N}}+µ^2 \\
\\
Grundlage\ III: \\
E(µ^2) &= {\frac{1}{N^2}}{\sum_{i,j = 1}^N{E(x_ix_j)}} \\
&= {\frac{1}{N^2}}({\sum_{i=1}^N{E(x_i^2)}} + {\sum_{j=1...N;\ (j\neq i)}{E(x_i)E(x_j)}}) \\
&= {\frac{1}{N^2}}({\sum_{i=1}^N{(Var(x_i)+E(x_i)^2)}}+ µ(N-1)µ)\\
&= {\frac{1}{N^2}}(N(Var(x_i)+E(x_i)^2)+N(N-1)µ^2) \\
&= {\frac{N Var(x_i)+N µ^2 + N^2µ^2-µ^2N}{N^2}} \\
&= {\frac{1}{N}}(Var(x_i)+µ^2+Nµ^2-µ^2) \\
&= {\frac{Var(x_i)}{N}}+µ^2
\\
\end{align}
$$

### b) Korrektur des Schätzers ###

Der Schätzer der Varianz ist 
$$
\begin{align}
s^2 = Var(x_i) &= {\frac{{\sum_{i=1}^N{(x_i-µ)^2}}}{N}} \\
\end{align}
$$
Aus Aufgabe a) geht hervor: 
$$
\begin{align}
E(Var(x_i)) &= Var(x_i) ({\frac{N-1}{N}}) \\
\end{align}
$$
Damit die Verzerrung des Schätzers eliminiert würde müsste 
$$
\begin{align}
E(Var(x_i)) &= Var(x_i) \\
\end{align}
$$
sein. 

#### Korrektur des Schätzers: 
$$
\begin{align}
Var(x_i) &= {\frac{{\sum_{i=1}^N{(x_i-µ)^2}}}{N}} ({\frac{N}{N-1}})\\
Var(x_i) &= {\frac{\sum_{i = 1}^N{(x_i - µ)^2}}{N-1}}
\end{align}
$$